## Generating Order for Round 1 Assessment
To prevent bias in assessing model quality, the order of few shot and zero shot models has been randomized. Additionally, the order of model ID's has also been randomized to prevent too many cases in the same domain appearing consecutively.

In [1]:
import random

# List of IDs
ids = [
    "FTP_1071",
    "FTP_1150",
    "CEM_435",
    "CEM_110",
    "IOPM_855",
    "IOPM_585",
    "CEM_975",
    "CEM_1000"
]

# Shuffle the list of IDs
random.shuffle(ids)

# For each ID, create the Zero-shot and Few-shot pair, and randomize the order inside the pair
final_order = []

for id_ in ids:
    pair = [f"Zero_shot_{id_}", f"Few_shot_{id_}"]
    random.shuffle(pair)  # Randomize within the pair
    final_order.extend(pair)

# Output the final randomized list
for idx, name in enumerate(final_order, 1):
    print(f"{idx}. {name}")

1. Zero_shot_CEM_110
2. Few_shot_CEM_110
3. Few_shot_FTP_1071
4. Zero_shot_FTP_1071
5. Zero_shot_IOPM_855
6. Few_shot_IOPM_855
7. Few_shot_FTP_1150
8. Zero_shot_FTP_1150
9. Zero_shot_CEM_1000
10. Few_shot_CEM_1000
11. Few_shot_CEM_975
12. Zero_shot_CEM_975
13. Few_shot_IOPM_585
14. Zero_shot_IOPM_585
15. Zero_shot_CEM_435
16. Few_shot_CEM_435


### Final Order Used in Round 1 
#### (above output might differ due to regeneration)
1. Few_shot_IOPM_855
2. Zero_shot_IOPM_855
3. Zero_shot_CEM_975
4. Few_shot_CEM_975
5. Zero_shot_FTP_1071
6. Few_shot_FTP_1071
7. Zero_shot_FTP_1150
8. Few_shot_FTP_1150
9. Zero_shot_CEM_1000
10. Few_shot_CEM_1000
11. Zero_shot_CEM_435
12. Few_shot_CEM_435
13. Zero_shot_IOPM_585
14. Few_shot_IOPM_585
15. Few_shot_CEM_110
16. Zero_shot_CEM_110

## Generating Order for Round 2 Assessment
Now for round 2, the expert is shown three different models for each case, two of which correspond to two human modelers, and one corresponds to the LLM generated model. The below code is used to randomly order the three different models to prevent bias.

In [20]:
import random
from pprint import pprint

# --- 1) list the eight cases --------------------------------------------------
cases = [
    "FTP_1071",
    "FTP_1150",
    "CEM_435",
    "CEM_110",
    "IOPM_855",
    "IOPM_585",
    "CEM_975",
    "CEM_1000",
]

# --- 2) list the three model flavours per case --------------------------------
models = ["Human_A", "Human_B", "LLM"]      

# ----------------------------------------------------------------------------- #
# Shuffle the list of IDs
random.shuffle(cases)

# Shuffle three model labels within each ID
final_sequence = []
for cid in cases:
    shuffled = models[:]          # local copy
    random.shuffle(shuffled)
    # prepend the case-ID to each model label for clarity
    final_sequence.extend([f"{cid}_{m}" for m in shuffled])

# Pretty-print the resulting 24-item order
pprint(final_sequence, width=80)


['IOPM_855_Human_A',
 'IOPM_855_Human_B',
 'IOPM_855_LLM',
 'CEM_435_Human_A',
 'CEM_435_LLM',
 'CEM_435_Human_B',
 'FTP_1071_Human_A',
 'FTP_1071_LLM',
 'FTP_1071_Human_B',
 'CEM_975_Human_B',
 'CEM_975_Human_A',
 'CEM_975_LLM',
 'IOPM_585_Human_B',
 'IOPM_585_Human_A',
 'IOPM_585_LLM',
 'CEM_110_LLM',
 'CEM_110_Human_B',
 'CEM_110_Human_A',
 'FTP_1150_Human_A',
 'FTP_1150_Human_B',
 'FTP_1150_LLM',
 'CEM_1000_Human_B',
 'CEM_1000_LLM',
 'CEM_1000_Human_A']


### Final Order Used in Round 2
#### (above output might differ due to regeneration)
'IOPM_855_Human_A',
 'IOPM_855_Human_B',
 'IOPM_855_LLM',
 'CEM_435_Human_A',
 'CEM_435_LLM',
 'CEM_435_Human_B',
 'FTP_1071_Human_A',
 'FTP_1071_LLM',
 'FTP_1071_Human_B',
 'CEM_975_Human_B',
 'CEM_975_Human_A',
 'CEM_975_LLM',
 'IOPM_585_Human_B',
 'IOPM_585_Human_A',
 'IOPM_585_LLM',
 'CEM_110_LLM',
 'CEM_110_Human_B',
 'CEM_110_Human_A',
 'FTP_1150_Human_A',
 'FTP_1150_Human_B',
 'FTP_1150_LLM',
 'CEM_1000_Human_B',
 'CEM_1000_LLM',
 'CEM_1000_Human_A

### Apply order to actual cases and save in final zip file
The zip file was extracted and uploaded to Google Drive to provide the experts with all models, without knowing their original label

In [27]:
import zipfile
from pathlib import Path

# ---- configuration -------------------------------------------------
zip_path = Path("user_bdd_sample_set.zip")      
extract_to = Path("unzipped_models")            
# --------------------------------------------------------------------

extract_to.mkdir(exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_to)

print(f"✅  Extracted {zip_path} into → {extract_to.resolve()}")

✅  Extracted user_bdd_sample_set.zip into → /home/jorick/documents/unzipped_models


In [30]:
import os, shutil, re
from pathlib import Path

src_roots = {
    "Human_A": "Alice",
    "Human_B": "Arne",
    "LLM"    : "LLM"
}

dst_root = Path("cases")          # where the reorganised tree will be written
png_pattern = re.compile(r".*\.png$", re.I)   # copy only PNGs
# -------------------------------------------------------------------------

dst_root.mkdir(exist_ok=True)

for label in final_sequence:
    m = re.match(r"(?P<id>.+)_(?P<author>Human_[AB]|LLM)$", label)
    if not m:
        raise ValueError(f"Label format unexpected: {label}")
    case_id   = m["id"]
    author    = m["author"]

    src_dir = Path("unzipped_models") / src_roots[author] / f"{case_id} visuals"
    if not src_dir.is_dir():
        print(f"⚠️  Source folder missing: {src_dir}")
        continue

    case_base = dst_root / case_id
    case_base.mkdir(exist_ok=True)

    existing = sorted(p for p in case_base.iterdir() if p.is_dir())
    next_letter = chr(ord("A") + len(existing))
    dst_dir = case_base / f"model_{next_letter}"
    dst_dir.mkdir()

    # Copy the two PNGs
    for f in src_dir.iterdir():
        if png_pattern.match(f.name):
            shutil.copy2(f, dst_dir / f.name)

    print(f"Copied {label} → {dst_dir}")

print("\nDone. Check the 'cases' folder.")


Copied IOPM_855_Human_A → cases/IOPM_855/model_A
Copied IOPM_855_Human_B → cases/IOPM_855/model_B
Copied IOPM_855_LLM → cases/IOPM_855/model_C
Copied CEM_435_Human_A → cases/CEM_435/model_A
Copied CEM_435_LLM → cases/CEM_435/model_B
Copied CEM_435_Human_B → cases/CEM_435/model_C
Copied FTP_1071_Human_A → cases/FTP_1071/model_A
Copied FTP_1071_LLM → cases/FTP_1071/model_B
Copied FTP_1071_Human_B → cases/FTP_1071/model_C
Copied CEM_975_Human_B → cases/CEM_975/model_A
Copied CEM_975_Human_A → cases/CEM_975/model_B
Copied CEM_975_LLM → cases/CEM_975/model_C
Copied IOPM_585_Human_B → cases/IOPM_585/model_A
Copied IOPM_585_Human_A → cases/IOPM_585/model_B
Copied IOPM_585_LLM → cases/IOPM_585/model_C
Copied CEM_110_LLM → cases/CEM_110/model_A
Copied CEM_110_Human_B → cases/CEM_110/model_B
Copied CEM_110_Human_A → cases/CEM_110/model_C
Copied FTP_1150_Human_A → cases/FTP_1150/model_A
Copied FTP_1150_Human_B → cases/FTP_1150/model_B
Copied FTP_1150_LLM → cases/FTP_1150/model_C
Copied CEM_1000_H

### Ensure File Names are Standardized across Modelers
Before the below step, each modeler still had different naming conventions for saving their UML Use Case diagram and BPMN files. Without renaming, this would allow identification of the modeler, leading to bias. 

In [31]:
import re
from pathlib import Path
from shutil import move

CASES_ROOT = Path("cases")          # adjust if you put it elsewhere
ID_PATTERN = re.compile(r"^([A-Z]+_\d+)(?:\[1\])?\.png$", re.IGNORECASE)

def canonical_name(original: Path) -> str | None:
    """
    Decide which canonical filename a given PNG should receive.
    Returns 'UML Use Case.png', 'BPMN.png', or None (leave unchanged).
    """
    m = ID_PATTERN.match(original.name)
    if not m:
        return None                       # already good or foreign file
    base_id = m.group(1)
    if original.name.endswith("[1].png"):
        return "BPMN.png"                 # e.g. CEM_110[1].png → BPMN.png
    else:
        return "UML Use Case.png"         # e.g. CEM_110.png → UML Use Case.png

def rename_images():
    for img in CASES_ROOT.rglob("*.png"):
        target_name = canonical_name(img)
        if target_name is None:
            continue                      # skip files that are already OK
        target_path = img.with_name(target_name)
        if target_path.exists():
            print(f"⚠️  {target_path} already exists – skipping {img.name}")
            continue
        move(img, target_path)
        print(f"✓  {img.relative_to(CASES_ROOT.parent)}  ➜  {target_path.name}")

if __name__ == "__main__":
    rename_images()


✓  cases/IOPM_855/model_A/IOPM_855.png  ➜  UML Use Case.png
✓  cases/IOPM_855/model_A/IOPM_855[1].png  ➜  BPMN.png
✓  cases/IOPM_855/model_B/IOPM_855.png  ➜  UML Use Case.png
✓  cases/CEM_975/model_A/CEM_975.png  ➜  UML Use Case.png
✓  cases/CEM_975/model_B/CEM_975.png  ➜  UML Use Case.png
✓  cases/CEM_975/model_B/CEM_975[1].png  ➜  BPMN.png
✓  cases/FTP_1150/model_A/FTP_1150[1].png  ➜  BPMN.png
✓  cases/FTP_1150/model_A/FTP_1150.png  ➜  UML Use Case.png
✓  cases/FTP_1150/model_B/FTP_1150.png  ➜  UML Use Case.png
✓  cases/CEM_435/model_A/CEM_435[1].png  ➜  BPMN.png
✓  cases/CEM_435/model_A/CEM_435.png  ➜  UML Use Case.png
✓  cases/CEM_435/model_C/CEM_435.png  ➜  UML Use Case.png
✓  cases/CEM_1000/model_A/CEM_1000.png  ➜  UML Use Case.png
✓  cases/CEM_1000/model_C/CEM_1000.png  ➜  UML Use Case.png
✓  cases/CEM_1000/model_C/CEM_1000[1].png  ➜  BPMN.png
✓  cases/FTP_1071/model_A/FTP_1071.png  ➜  UML Use Case.png
✓  cases/FTP_1071/model_C/FTP_1071.png  ➜  UML Use Case.png
✓  cases/CEM_110/

In [32]:
from pathlib import Path
import shutil

source_dir = Path("cases")       
assert source_dir.exists(), f"{source_dir} does not exist."

zip_stem = "cases_for_drive"       

archive_path = shutil.make_archive(zip_stem, format="zip", root_dir=source_dir)

print(f"Archive created at: {archive_path}")

Archive created at: /home/jorick/documents/cases_for_drive.zip
